In [1]:
import numpy as np
from collections import deque
from tqdm import trange
from icecream import ic

def cost(sol):
    # checks each column (each item) across all knapsacks
    # returns a boolean vector of length NUM_ITEMS, where each entry is True if that item appears in at least one knapsack
    # and sum values of selected items
    return VALUES[np.any(sol, axis=0)].sum()

def is_feasible(weights):
    return np.all(weights <= CONSTRAINTS)

def random_solution():
    sol = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
    total_weights = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS))
    for i in np.random.permutation(NUM_ITEMS):
        k = np.random.randint(NUM_KNAPSACKS)
        if np.all(total_weights[k] + WEIGHTS[i] <= CONSTRAINTS[k]):
            sol[k, i] = True
            total_weights[k] += WEIGHTS[i]
    value = cost(sol)
    return sol, total_weights, value

def tabu_search(
    max_iter=5000,
    tabu_tenure=50,
    neighborhood_size=200,
    no_move_limit=300,
    restart_limit=600,
):
    sol, total_weights, best_val = random_solution()
    best_sol = sol.copy()
    current_val = best_val
    tabu_list = deque(maxlen=tabu_tenure)
    no_move_counter = 0
    
    for it in trange(max_iter, desc="Tabu Search"):
        best_neighbor = None
        best_neighbor_val = -np.inf
        best_neighbor_weights = None
        best_move = None
        # generate neighborhood 
        for _ in range(neighborhood_size):
            i = np.random.randint(NUM_ITEMS)
            # check if item i is already assigned to a knapsack
            old_k = np.where(sol[:, i])[0]
            old_k = old_k[0] if len(old_k) > 0 else None
            k = np.random.randint(-1, NUM_KNAPSACKS) # new knapsack (-1 = remove from solution)
            if k == old_k: # skip is there is no actual change
                continue
            # check feasibility
            if k != -1 and np.any(total_weights[k] + WEIGHTS[i] > CONSTRAINTS[k]):
                continue # infeasible, skip
            
            move = (i, old_k, k)
            if move in tabu_list:
                continue # skip
            
            # apply move
            new_sol = sol.copy()
            new_weights = total_weights.copy()
            i, old_k, k = move
            if old_k is not None: # remove item from old knapsack
                new_sol[old_k, i] = False
                new_weights[old_k] -= WEIGHTS[i]
            if k != -1: # add item to new knapsack
                new_sol[k, i] = True
                new_weights[k] += WEIGHTS[i]
            # evaluate solution
            new_val = cost(new_sol)          
            if new_val > best_neighbor_val:
                best_neighbor = new_sol
                best_neighbor_val = new_val
                best_neighbor_weights = new_weights
                best_move = move

        # if no improvement in the neighborhood
        if best_neighbor is None:
            no_move_counter += 1
            # after a while, restart from another random solution
            if no_move_counter > restart_limit:
                sol, total_weights, current_val = random_solution()
                tabu_list.clear()
                tabu_tenure = max(10, tabu_tenure // 2)
                no_move_counter = 0
            continue

        # else, apply best move
        sol = best_neighbor
        total_weights = best_neighbor_weights
        current_val = best_neighbor_val
        tabu_list.append(best_move) 
        # update best
        if current_val > best_val:
            best_val = current_val
            best_sol = sol.copy()
            no_move_counter = 0
        else:
            no_move_counter += 1

        # dynamic tabu adjustment
        if no_move_counter > no_move_limit:
            tabu_tenure = max(10, tabu_tenure // 2)
        else:
            tabu_tenure = min(100, tabu_tenure + 1)

    return best_sol, best_val


## TEST PROBLEMS

In [2]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
max_theoretical = VALUES.sum()
print("theoretical max:", max_theoretical)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# ---------- Run ---------- #
best_sol, best_val = tabu_search(max_iter=100)
ic(best_sol.astype(int))
ic(best_val)

theoretical max: 1065


Tabu Search: 100%|██████████| 100/100 [00:00<00:00, 665.66it/s]
ic| best_sol.astype(int): array([[0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0],
                                 [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
                                 [1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]])
ic| best_val: 1065


1065

In [3]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
max_theoretical = VALUES.sum()
print("theoretical max:", max_theoretical)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# ---------- Run ---------- #
best_sol, best_val = tabu_search(max_iter=5000)
# ic(best_sol.astype(int))
ic(best_val)

theoretical max: 52620


Tabu Search: 100%|██████████| 5000/5000 [00:06<00:00, 777.74it/s]
ic| best_val: 50768


50768

In [17]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
max_theoretical = VALUES.sum()
print("theoretical max (without constraints):", max_theoretical)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# ---------- Run ---------- #
best_sol, best_val = tabu_search(max_iter=50_000)
ic(best_sol.astype(int))
ic(best_val)

theoretical max (without constraints): 2490698


Tabu Search: 100%|██████████| 50000/50000 [01:16<00:00, 650.92it/s]
ic| best_sol.astype(int): array([[0, 0, 0, ..., 0, 0, 0],
                                 [0, 0, 0, ..., 0, 0, 0],
                                 [0, 0, 0, ..., 0, 0, 0],
                                 ...,
                                 [0, 0, 0, ..., 0, 0, 0],
                                 [0, 0, 0, ..., 0, 0, 0],
                                 [0, 0, 0, ..., 0, 0, 0]])
ic| best_val: 1786794


1786794